In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import sklearn
import scipy.stats as stats
import pydotplus
import seaborn as sns; sns.set()

In [ ]:
#Exo A1
#Question 1
train = pd.read_csv('titanic_train.csv')
test = pd.read_csv('titanic_test.csv')

In [ ]:
train.head()

In [ ]:
train.Age.isnull()

In [ ]:
train.sum()

In [ ]:
train_headers = train.columns.tolist()
for i in range(len(train_headers)):
    x = train_headers[i]
    y = pd.isnull(train[x]).value_counts()
    print(y)
    print('type', np.dtype(train[x]))
    print()

In [ ]:
age_drop = train.dropna(subset=['Age'])

In [ ]:
#Question 2
plt.title("Histogramme de l'âge des passagers")
plt.xlabel("Âge")
plt.ylabel("Nombre de passagers")
plt.hist(age_drop.Age, edgecolor="black")

In [ ]:
(train.shape[0]-train.Survived.sum())*100/train.shape[0]

In [ ]:
train.Pclass.value_counts()

In [ ]:
plt.title("Diagramme en camembert pour les classes")
plt.pie(train.Pclass.value_counts(), labels=['3ème', '1ère', '2nde'], autopct='%1.1f%%')

In [ ]:
train.Survived[train.Sex=="male"].value_counts()

In [ ]:
train.Survived[train.Sex=="female"].value_counts()

In [ ]:
train.Survived.value_counts()

In [ ]:
train.Sex.value_counts()[1]*100/(train.Sex.value_counts()[0]+train.Sex.value_counts()[1])

In [ ]:
train.Survived[train.Sex=="female"].value_counts()[1]*100/(train.Survived[train.Sex=="female"].value_counts()[0]+train.Survived[train.Sex=="female"].value_counts()[1])

In [ ]:
age_drop.Survived[age_drop.Age>=18].value_counts()

In [ ]:
age_drop.Survived[age_drop.Age>=18].value_counts()[1]*100/(age_drop.Survived[age_drop.Age>=18].value_counts()[0]+age_drop.Survived[age_drop.Age>=18].value_counts()[1])

In [ ]:
age_drop.Survived[age_drop.Age>=18][age_drop.Sex=="male"].value_counts()

In [ ]:
train.Survived[train.Pclass==3].value_counts()

In [ ]:
sns.heatmap(train.corr(), annot=True, linewidth=2)

In [ ]:
#Question 5
newfare = train.Fare.copy()
tempfare = train.Fare.copy()
for i in range(582):
    if tempfare[i]<10:
        newfare[i]=1
    if tempfare[i]<20 and tempfare[i]>=10:
        newfare[i]=2
    if tempfare[i]<30 and tempfare[i]>=20:
        newfare[i]=3
    if tempfare[i]>=30:
        newfare[i]=4
train.insert(13, "Fare2", newfare)
print(train)
sns.heatmap(train.corr(), annot=True, linewidth=1)

In [ ]:
newfare = test.Fare.copy()
tempfare = test.Fare.copy()
for i in range(309):
    if tempfare[i]<10:
        newfare[i]=1
    if tempfare[i]<20 and tempfare[i]>=10:
        newfare[i]=2
    if tempfare[i]<30 and tempfare[i]>=20:
        newfare[i]=3
    if tempfare[i]>=30:
        newfare[i]=4
test.insert(13, "Fare2", newfare)
print(train)
sns.heatmap(test.corr(), annot=True, linewidth=1,)

In [ ]:
train.Survived[train.Fare2==3].value_counts()

In [ ]:
print(stats.chi2_contingency(pd.crosstab(train.Fare2, train.Pclass)))

In [ ]:
#Exo A2
#Question 1
newage = train.Age.copy()
tempage = train.Age.copy()
newsex = train.Sex.copy()
tempsex = train.Sex.copy()

for i in range(582):
    if tempage[i]<18:
        newage[i]=1
    if tempage[i]>=18:
        newage[i]=0
    if tempsex[i].__eq__('female'):
        newsex[i]=0
    if tempsex[i].__eq__('male'):
        newsex[i]=1
train.insert(14, "Child", newage)
train.insert(15, "Sex_cat", newsex)

age_drop = train.dropna(subset=['Age'])

newage = test.Age.copy()
tempage = test.Age.copy()
newsex = test.Sex.copy()
tempsex = test.Sex.copy()

for i in range(309):
    if tempage[i]<18:
        newage[i]=1
    if tempage[i]>=18:
        newage[i]=0
    if tempsex[i].__eq__('female'):
        newsex[i]=0
    if tempsex[i].__eq__('male'):
        newsex[i]=1
test.insert(14, "Child", newage)
test.insert(15, "Sex_cat", newsex)

age_drop_test = test.dropna(subset=['Age'])

In [ ]:
#Question 2
from sklearn import metrics
from sklearn.naive_bayes import GaussianNB

gnbModel= GaussianNB()

gnbModel.fit(age_drop[['Child', 'Sex_cat']], age_drop['Fare2'])

expected = age_drop['Fare2']

predicted = gnbModel.predict(age_drop[['Child', 'Sex_cat']])
print(metrics.classification_report(expected, predicted))

expected = age_drop_test['Fare2']
predicted = gnbModel.predict(age_drop_test[['Child', 'Sex_cat']])
print(metrics.classification_report(expected, predicted))

In [ ]:
#Exo A3
import pydotplus
import collections
from IPython.display import Image
from sklearn.tree import export_graphviz
from sklearn import tree

clf = tree.DecisionTreeClassifier()
clf = clf.fit(age_drop[['Child', 'Sex_cat']],
              age_drop['Survived'])

dot_data = tree.export_graphviz(clf, out_file=None,
                                feature_names=['Child', 'Sex_cat'],
                                class_names=["DCeased", "Alive"],
                                filled=True,
                                rounded=True)
graph = pydotplus.graph_from_dot_data(dot_data)
Image(graph.create_png())
Image(graph.write_png('./filename.png'))